In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
import seaborn as sns
from scipy.stats import ttest_rel
from sklearn.model_selection import cross_val_score

from utils import *

%load_ext autoreload
%autoreload 2
sns.set_context("talk")


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [37]:
DATA_PATH = Path("/Users/jessbreda/Desktop/github/ca-sdoh/data")

cols_of_interest = [
    "condition_name",
    "number_of_readmissions",
    "number_of_discharges",
    "avg_medicare_beneficiaries_py",
    "payment",
    "beneficiary_avg_age",
    "avg_number_female_beneficiaries_py",
    "pct_beneficiaries_afib",
    "pct_beneficiaries_alzheimers",
    "pct_beneficiaries_cancer",
    "pct_beneficiaries_chronic_kidney_disease",
    "pct_beneficiaries_congestive_heart_failure",
    "pct_beneficiaries_ischemic_heart_disease",
    "pct_beneficiaries_copd",
    "pct_beneficiaries_depression",
    "pct_beneficiaries_diabetes",
    "pct_beneficiaries_osteoporosis",
    "pct_beneficiaries_arthritis",
    "pct_beneficiaries_psychotic_disorders",
    "pct_beneficiaries_stroke",
    "avg_number_beneficiaries_medicare_medicaid_py",
    "avg_number_beneficiaries_medicare_only_py",
    "avg_number_white_beneficiaries_py",
    "avg_number_black_beneficiaries_py",
    "avg_number_hispanic_beneficiaries_py",
]

df = pd.read_csv(
    (DATA_PATH / "obt_slim.csv"),
    usecols=cols_of_interest,
)

# filter for heart failure
df = df[df["condition_name"] == "Heart Failure"]

# drop nans
df.dropna(inplace=True)

# calculate readmission rate & drop the original columns
df["readmission_rate"] = (
    df["number_of_readmissions"] / df["number_of_discharges"]
) * 100
df.drop(
    columns=["number_of_readmissions", "number_of_discharges", "condition_name"],
    inplace=True,
)

df["pct_female"] = (
    100 * df["avg_number_female_beneficiaries_py"] / df["avg_medicare_beneficiaries_py"]
)

df["pct_white"] = (
    100 * df["avg_number_white_beneficiaries_py"] / df["avg_medicare_beneficiaries_py"]
)
df["pct_black"] = (
    100 * df["avg_number_black_beneficiaries_py"] / df["avg_medicare_beneficiaries_py"]
)
df["pct_hispanic"] = (
    100
    * df["avg_number_hispanic_beneficiaries_py"]
    / df["avg_medicare_beneficiaries_py"]
)
df["pct_medicare_medicaid"] = (
    100
    * df["avg_number_beneficiaries_medicare_medicaid_py"]
    / df["avg_medicare_beneficiaries_py"]
)
df["pct_medicare_only"] = (
    100
    * df["avg_number_beneficiaries_medicare_only_py"]
    / df["avg_medicare_beneficiaries_py"]
)
df["payment"] = df.payment.str.replace("$", "").str.replace(",", "").astype(float)

X = df.drop(
    columns=[
        "avg_medicare_beneficiaries_py",
        "avg_number_female_beneficiaries_py",
        "avg_number_white_beneficiaries_py",
        "avg_number_black_beneficiaries_py",
        "avg_number_hispanic_beneficiaries_py",
        "avg_number_beneficiaries_medicare_medicaid_py",
        "avg_number_beneficiaries_medicare_only_py",
        "readmission_rate",
    ]
)
race_cols = [
    "pct_black",
    "pct_white",
    "pct_hispanic",
]
y = df.readmission_rate
X_no_race = X.drop(columns=race_cols)
X.reset_index(drop=True, inplace=True)
X_no_race.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)

In [46]:
# Train_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Create the Random Forest regressor
rf = RandomForestRegressor(
    n_estimators=100, random_state=42
)  # You can adjust n_estimators as needed.

# Fit the model to the training data
rf.fit(X_train, y_train)

y_pred = rf.predict(X_test)

# Calculate mean squared error (MSE) and R-squared (R2)
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print(f"Mean Squared Error: {mse}")
print(f"R-squared: {r2}")

# Get the feature importances
importances = rf.feature_importances_

# Normalize the importances to sum up to 1
importances_normalized = importances / np.sum(importances)

# Sort the features based on their importance values
sorted_indices = np.argsort(importances_normalized)[::-1]

# Print the feature names and their corresponding importances in descending order
for i in sorted_indices:
    print(f"{X_train.columns[i]}: {importances_normalized[i]}")

Mean Squared Error: 11.411241635909121
R-squared: 0.3352508030476796
pct_beneficiaries_diabetes: 0.15536260330391985
pct_beneficiaries_psychotic_disorders: 0.10267345424795477
pct_beneficiaries_ischemic_heart_disease: 0.051747905540945514
pct_black: 0.049539472586901036
payment: 0.04845976640689681
pct_beneficiaries_copd: 0.048382440790147065
pct_beneficiaries_stroke: 0.04585321653333613
pct_beneficiaries_afib: 0.04248214953217564
pct_female: 0.04034778408924461
pct_beneficiaries_cancer: 0.03995859123974891
pct_beneficiaries_osteoporosis: 0.038275075806158874
pct_beneficiaries_arthritis: 0.03824848523035798
pct_beneficiaries_alzheimers: 0.037971883327476245
pct_beneficiaries_congestive_heart_failure: 0.037026644979716035
pct_beneficiaries_depression: 0.036999629283140247
beneficiary_avg_age: 0.03621115016255172
pct_beneficiaries_chronic_kidney_disease: 0.03610380032123259
pct_medicare_medicaid: 0.032376003691320016
pct_white: 0.028836542519619028
pct_hispanic: 0.028550677293543353
pct_

In [47]:
# Perform cross-validation with 5 folds (you can adjust the number of folds as needed)
num_folds = 5
cv_scores = cross_val_score(
    rf, X_train, y_train, cv=num_folds, scoring="neg_mean_squared_error"
)

# Since 'cross_val_score' returns negative MSE, we need to take the absolute value and calculate RMSE.
rmse_scores = np.sqrt(np.abs(cv_scores))

# Print the RMSE scores for each fold
print("RMSE Scores for each fold:")
print(rmse_scores)

# Print the average RMSE across all folds
print("Average RMSE:", np.mean(rmse_scores))

RMSE Scores for each fold:
[3.56229365 3.25467266 3.74617697 3.2644931  3.4483154 ]
Average RMSE: 3.4551903538472772


In [43]:
from sklearn.model_selection import KFold

rf = RandomForestRegressor(n_estimators=100, random_state=32)

# Perform cross-validation with 5 folds (you can adjust the number of folds as needed)
num_folds = 5

# Initialize an array to store feature importance rankings for each fold
feature_ranks = np.zeros((num_folds, X.shape[1]))
kf = KFold(n_splits=num_folds, shuffle=True, random_state=32)

# Perform cross-validation and record feature importances for each fold
for fold_idx, (train_idx, test_idx) in enumerate(kf.split(X)):
    X_train, X_test = X.iloc[train_idx], X.iloc[test_idx]
    y_train, y_test = y.iloc[train_idx], y.iloc[test_idx]

    # Fit the model to the training data
    rf.fit(X_train, y_train)

    # Get the feature importances and rank them for this fold
    importances = rf.feature_importances_
    ranked_indices = np.argsort(importances)[::-1]
    feature_ranks[fold_idx] = ranked_indices

# Calculate the consistency of feature importance rankings across folds
consistency_scores = np.zeros(X.shape[1])
for feature_idx in range(X.shape[1]):
    unique_rankings, counts = np.unique(
        feature_ranks[:, feature_idx], return_counts=True
    )
    consistency_scores[feature_idx] = np.max(counts) / num_folds

# Print the feature names and their consistency scores in descending order
sorted_indices = np.argsort(consistency_scores)[::-1]
for i in sorted_indices:
    print(f"{X.columns[i]}: {consistency_scores[i]}")

pct_medicare_medicaid: 0.8
pct_medicare_only: 0.6
beneficiary_avg_age: 0.6
pct_beneficiaries_alzheimers: 0.6
pct_beneficiaries_ischemic_heart_disease: 0.6
payment: 0.6
pct_female: 0.4
pct_beneficiaries_stroke: 0.4
pct_beneficiaries_arthritis: 0.4
pct_beneficiaries_osteoporosis: 0.4
pct_white: 0.4
pct_beneficiaries_diabetes: 0.4
pct_beneficiaries_copd: 0.4
pct_beneficiaries_cancer: 0.4
pct_black: 0.4
pct_beneficiaries_afib: 0.4
pct_beneficiaries_psychotic_disorders: 0.2
pct_beneficiaries_depression: 0.2
pct_beneficiaries_chronic_kidney_disease: 0.2
pct_beneficiaries_congestive_heart_failure: 0.2
pct_hispanic: 0.2


In [32]:
DATA_PATH = Path("/Users/jessbreda/Desktop/github/ca-sdoh/data")

cols_of_interest = [
    "condition_name",
    "number_of_readmissions",
    "number_of_discharges",
    "avg_medicare_beneficiaries_py",
    "payment",
    "beneficiary_avg_age",
    "avg_number_female_beneficiaries_py",
    "pct_beneficiaries_afib",
    "pct_beneficiaries_alzheimers",
    "pct_beneficiaries_cancer",
    "pct_beneficiaries_chronic_kidney_disease",
    "pct_beneficiaries_congestive_heart_failure",
    "pct_beneficiaries_ischemic_heart_disease",
    "pct_beneficiaries_copd",
    "pct_beneficiaries_depression",
    "pct_beneficiaries_diabetes",
    "pct_beneficiaries_osteoporosis",
    "pct_beneficiaries_arthritis",
    "pct_beneficiaries_psychotic_disorders",
    "pct_beneficiaries_stroke",
    "avg_number_beneficiaries_medicare_medicaid_py",
    "avg_number_beneficiaries_medicare_only_py",
]

df = pd.read_csv(
    (DATA_PATH / "obt_slim.csv"),
    usecols=cols_of_interest,
)

# filter for heart failure
df = df[df["condition_name"] == "Heart Failure"]

# drop nans
df.dropna(inplace=True)

# calculate readmission rate & drop the original columns
df["readmission_rate"] = (
    df["number_of_readmissions"] / df["number_of_discharges"]
) * 100
df.drop(
    columns=["number_of_readmissions", "number_of_discharges", "condition_name"],
    inplace=True,
)

df["pct_female"] = (
    100 * df["avg_number_female_beneficiaries_py"] / df["avg_medicare_beneficiaries_py"]
)
df["pct_medicare_medicaid"] = (
    100
    * df["avg_number_beneficiaries_medicare_medicaid_py"]
    / df["avg_medicare_beneficiaries_py"]
)
df["pct_medicare_only"] = (
    100
    * df["avg_number_beneficiaries_medicare_only_py"]
    / df["avg_medicare_beneficiaries_py"]
)
df["payment"] = df.payment.str.replace("$", "").str.replace(",", "").astype(float)

X = df.drop(
    columns=[
        "avg_medicare_beneficiaries_py",
        "avg_number_female_beneficiaries_py",
        "avg_number_beneficiaries_medicare_medicaid_py",
        "avg_number_beneficiaries_medicare_only_py",
        "readmission_rate",
    ]
)

y = df.readmission_rate

X.reset_index(drop=True, inplace=True)
y.reset_index(drop=True, inplace=True)

# Train_test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

In [41]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import mean_squared_error, r2_score

X_train1, X_test1, y_train, y_test = train_test_split(X, y, test_size=0.2)

X_train2, X_test2 = X_train1[X_no_race.columns], X_test1[X_no_race.columns]

# Assuming you have your feature matrices X_train1, X_train2 and target vectors y_train1, y_train2 ready
# Create the first Random Forest regressor with subset of features
rf1 = RandomForestRegressor(n_estimators=100, random_state=42)

# Create the second Random Forest regressor with a different subset of features
rf2 = RandomForestRegressor(n_estimators=100, random_state=42)

# Perform cross-validation with 5 folds (you can adjust the number of folds as needed)
num_folds = 5

# Cross-validation for the first model
y_pred1 = cross_val_predict(rf1, X_train1, y_train, cv=num_folds)
mse1 = mean_squared_error(y_train, y_pred1)
rmse1 = np.sqrt(mse1)
r2_1 = r2_score(y_train, y_pred1)

# Cross-validation for the second model
y_pred2 = cross_val_predict(rf2, X_train2, y_train, cv=num_folds)
mse2 = mean_squared_error(y_train, y_pred2)
rmse2 = np.sqrt(mse2)
r2_2 = r2_score(y_train, y_pred2)

# Print the performance metrics for each model
print("Model 1:")
print("RMSE:", rmse1)
print("R2:", r2_1)

print("Model 2 (no race):")
print("RMSE:", rmse2)
print("R2:", r2_2)

Model 1:
RMSE: 3.4206278053244397
R2: 0.25839845634305925
Model 2 (no race):
RMSE: 3.419268323899951
R2: 0.25898781784862124


In [49]:
import numpy as np
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error

# Assuming you have your feature matrix X and target vector y ready

# Create the Random Forest regressor
rf = RandomForestRegressor(n_estimators=100, random_state=56)
rf.fit(X, y)

# Predict on the original data
y_pred_original = rf.predict(X)

# Calculate the baseline mean squared error (MSE)
baseline_mse = mean_squared_error(y, y_pred_original)

# Initialize an array to store feature importances
permutation_importances = np.zeros(X.shape[1])

# Calculate permutation feature importances
for feature_idx in range(X.shape[1]):
    # Create a copy of the original feature values
    X_permuted = X.copy()
    # Shuffle the values of the selected feature
    X_permuted.iloc[:, feature_idx] = np.random.permutation(
        X_permuted.iloc[:, feature_idx]
    )

    # Predict using the model with the permuted feature
    y_pred_permuted = rf.predict(X_permuted)

    # Calculate the mean squared error with the permuted feature
    permuted_mse = mean_squared_error(y, y_pred_permuted)

    # Calculate the feature importance as the difference between the baseline MSE and permuted MSE
    permutation_importances[feature_idx] = baseline_mse - permuted_mse

# Normalize the importances by dividing by the standard deviation
permutation_importances /= np.std(permutation_importances)

# Print the feature names and their permutation feature importances in descending order
sorted_indices = np.argsort(permutation_importances)[::-1]
for i in sorted_indices:
    print(f"{X.columns[i]}: {permutation_importances[i]}")

pct_white: -0.5009703052254668
pct_hispanic: -0.5242592073401768
pct_medicare_only: -0.5701100603395572
pct_beneficiaries_congestive_heart_failure: -0.632243428677475
pct_beneficiaries_depression: -0.6586845572619976
pct_beneficiaries_arthritis: -0.7066933268172466
pct_beneficiaries_osteoporosis: -0.7073875864488219
pct_beneficiaries_stroke: -0.7689025953058181
pct_beneficiaries_chronic_kidney_disease: -0.7708910495239841
pct_female: -0.8444503329053753
pct_beneficiaries_alzheimers: -0.8558080876266881
pct_medicare_medicaid: -0.9034471276000461
beneficiary_avg_age: -0.9989363001747726
pct_beneficiaries_cancer: -1.028161154201089
pct_beneficiaries_afib: -1.028821689797929
pct_black: -1.3425205232493358
pct_beneficiaries_ischemic_heart_disease: -1.4840306317777516
payment: -1.5745837079089975
pct_beneficiaries_copd: -1.7648552086755618
pct_beneficiaries_psychotic_disorders: -3.7885750383694496
pct_beneficiaries_diabetes: -4.4429290324750905
